In [1]:
import gzip
import numpy
import pickle

import theano
from theano import tensor as T
from theano.tensor.nnet import conv2d

theano.config.exception_verbosity='high'
rng = numpy.random.RandomState(23455)

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [2]:
def get_convolutionaln_function(filters_number, filter_width, filter_height, input_depth):
    input_ = T.tensor4(name='input')
    w_shp = (filters_number, input_depth, filter_width, filter_height)
    #if weights are boud
    w_bound = numpy.sqrt(input_depth * filter_width * filter_height)
    W = theano.shared( numpy.asarray(
                rng.uniform(
                    low=-1.0 / w_bound,
                    high=1.0 / w_bound,
                    size=w_shp),
                dtype=input.dtype), name ='W')
    b_shp = (2,)
    b = theano.shared(numpy.asarray(
                rng.uniform(low=-.5, high=.5, size=b_shp),
                dtype=input.dtype), name ='b')
    # build symbolic expression that computes the convolution of input with filters in w
    conv_out = conv2d(input_, W)
    return T.nnet.sigmoid(conv_out + b.dimshuffle('x', 0, 'x', 'x'))

In [3]:
def shared_dataset(data_xy, borrow=True):
    data_x, data_y = data_xy
    shared_x = theano.shared(numpy.asarray(data_x,
                                           dtype=theano.config.floatX),
                             borrow=borrow)
    shared_y = theano.shared(numpy.asarray(data_y,
                                           dtype=theano.config.floatX),
                             borrow=borrow)
    return shared_x, T.cast(shared_y, 'int32')

            

def load_data(path='/home/siak/Data/MNIST/mnist.pkl.gz'):
    # Load the dataset
    with gzip.open(path, 'rb') as f:
        train_set, valid_set, test_set = pickle.load(f)

   
    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(valid_set)
    train_set_x, train_set_y = shared_dataset(train_set)

    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    return rval

In [4]:
class ConvLayer(object):
    """Pool Layer of a convolutional network """

    def __init__(self, rng, input_, filter_shape, input_shape):
        """
        :type filter_shape: tuple or list of length 4
        :param filter_shape: (number of output feature maps, num input feature maps,
                              filter height, filter width)

        :type input_shape: tuple or list of length 4
        :param input_shape: (batch size, num input feature maps,
                             image height, image width)
        """

        assert input_shape[1] == filter_shape[1]
        self.input_ = input_
        number_of_inputs = numpy.prod(filter_shape[1:])
        number_of_outputs = numpy.prod(filter_shape[0] + numpy.prod(filter_shape[2:]))

        self.W = theano.shared(
            numpy.asarray(rng.uniform(low=-1, high=1, size=filter_shape),
                dtype=theano.config.floatX
            ),
            borrow=True
        )

        # the bias is a 1D tensor -- one bias per output feature map
        b_values = numpy.zeros((filter_shape[0],), dtype=theano.config.floatX)
        self.b = theano.shared(value=b_values, borrow=True)

        # convolve input feature maps with filters
        conv_out = conv2d(
            input=input_,
            filters=self.W,
            filter_shape=filter_shape,
            image_shape=input_shape
        )


        # add the bias term. Since the bias is a vector (1D array), we first
        # reshape it to a tensor of shape (1, n_filters, 1, 1). Each bias will
        # thus be broadcasted across mini-batches and feature map
        # width & height
        self.output = conv_out + self.b.dimshuffle('x', 0, 'x', 'x')

        # store parameters of this layer
        self.params = [self.W, self.b]

        # keep track of model input
        self.input_ = input_
        
        
#remember to add zero padding
class PoolLayer(object):
    def __init__(self, rng, input_, poolsize=(2, 2)):
        self.output = downsample.max_pool_2d(
            input=input_,
            ds=poolsize,
            ignore_border=True
        )
        self.input_ = input_
        
class ReluLayer(object):
    def __init__(self, input_):
        self.output = T.tanh(input_)
        self.input_ = input_

In [75]:
class ConvoNet(object):
    def initialize_datasets(self):
        datasets = load_data()
        self.train_set_x, self.train_set_y = datasets[0]
        self.valid_set_x, self.valid_set_y = datasets[1]
        self.test_set_x, self.test_set_y = datasets[2]

        self.n_train_batches = self.train_set_x.get_value(borrow=True).shape[0]
        self.n_valid_batches = self.valid_set_x.get_value(borrow=True).shape[0]
        self.n_test_batches = self.test_set_x.get_value(borrow=True).shape[0]
        self.n_train_batches //= self.batch_size
        self.n_valid_batches //= self.batch_size
        self.n_test_batches //= self.batch_size
    
    def __init__(self, batch_size=500):
        self.state = numpy.random.RandomState(23455)
        self.index = T.lscalar()  # index to a [mini]batch
        self.batch_size = batch_size
        self.initialize_datasets()
        
        
        x = T.matrix('x')   # the data is presented as rasterized images
        #y = T.ivector('y')
        
        input_shape = (batch_size, 1, 28, 28)
        layer0_input = x.reshape(input_shape)
        self.layer0 = ConvLayer(self.state, input_=layer0_input, filter_shape=(16, 1, 5, 5), input_shape=input_shape)
        
        self.layer1 = ReluLayer(input_=self.layer0.output)
        self.input_data = self.test_set_x[self.index * batch_size: (self.index + 1) * self.batch_size]
        
        self.test_model = theano.function(
            [self.index],
            self.layer1.output,
            givens={
                x: self.test_set_x[self.index * batch_size: (self.index + 1) * self.batch_size],
                #y: self.test_set_y[self.index * batch_size: (self.index + 1) * self.batch_size]
        })
        
        


In [76]:
cn = ConvoNet()

In [11]:
out = cn.test_model(0)

In [77]:
from matplotlib import pyplot as plt
import PIL
from PIL import Image

out = cn.test_model(2)
im = Image.fromarray(out[0][0])

for idx, (x, y) in zip(range(16), window(range(0, 32))):
    im = Image.fromarray(out[0][idx] * 256).resize((48, 48), PIL.Image.ANTIALIAS)
    fltr = Image.fromarray(cn.layer0.W.get_value()[idx][0] * 256)
    fltr.resize((50, 50), PIL.Image.ANTIALIAS)
    plt.subplot(16, 2, x+1); plt.axis('off'); plt.imshow(fltr)
    plt.subplot(16, 2, y+1); plt.axis('off'); plt.imshow(im)
    
plt.show()

In [69]:
def window(l):
    it = iter(l)
    for x in it:
        yield (x, next(it))

def special_get_value(x):
    y = T.cast(x, 'int32')
    return y.eval()
    